# Assignment 1 - Intelligent Machines, Ethics and Law (COMP2400/6400)

### Name:

### Student Id:

# Facial Recognition

In this assignment, you will implement a **facial recognition system** using Python and explore its social implications. This assessment is designed to familiarize you with the process of training a facial recognition model, implementing it using a Python program; and critically examine the impact of facial recognition systems on society. This assignment will be marked out of 100, and will count towards **25%** of the total unit assessment.

## Olivetti Faces Dataset Description

The Olivetti Faces dataset is a popular dataset for testing and evaluating face recognition algorithms. It was created by AT&T Laboratories Cambridge and contains a set of grayscale images of faces, captured under different lighting conditions, facial expressions, and poses.

The dataset contains a total of **400 images**, each of size **64 x 64 pixels**. There are 10 different images for each of 40 distinct individuals, with each individual photographed in different sessions. The images were taken with a Canon camera, under controlled conditions that included a uniform background and controlled lighting.

The Olivetti Faces dataset is often used for tasks such as face recognition, face detection, and facial expression analysis. The dataset is popular because it is relatively small, making it easy to work with, and because it contains images that capture a wide range of facial variations.

The dataset is available in the scikit-learn Python library, which makes it easy to load and use for machine learning tasks. In scikit-learn, the Olivetti Faces dataset is provided as a dictionary with two keys: data and images. The data key contains a 2D array of the flattened image data, while the images key contains a 3D array of the original image data.


## Task 0 - (0 Marks)

Load the dataset using Python. How many images are in the dataset? 			

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA


# Load the dataset
faces = datasets.fetch_olivetti_faces()
data = faces.data       # Input data or input features
target = faces.target   # Label or Class or Variable to be predicted

# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=100, whiten=True)
X = pca.fit_transform(data)              ## X Represents the data and you will use for model training

# Size of the datset  
X.shape     # (Number of images, input feature)

## Task 1: Data Visualisation - (10 marks)

Visualize some (at least five) of the images in the dataset. Are there any noticeable differences between the images of different individuals? (Hint: You can use imshow() method to visualise images)

In [ ]:
# your code will go here

## Task 2: Training a Traditional Machine Learning Model	- (25 Marks)

Train a machine learning model for facial recognition with a machine learning algorithm discussed during the lectures **(Week 2 to Week 4)** (or any other appropriate algorithm not covered in the lectures). Which machine learning algorithm did you choose, and why? What is the accuracy of the model on the test set? **(Use 80% of the data for Training and 20% for testing)**

In [ ]:
# your code will go here

## Task 3: Training a Neural Network-based Model - (25 Marks)

Train an artificial neural network (Multi-Layered Perceptron Classifier (MLP)) model for facial recognition. What is the accuracy of the model on the test set? **(Use 80% of the data for Training and 20% for testing)**

In [ ]:
# your code will go here

## Task 4: Comparison - (20 Marks)

Compare the performance of the traditional machine learning model and the artificial neural network (MLP). Which model performed better, and why? Did the use of MLP instead of traditional machine learning algorithms involve any tradeoff(s)? If so, briefly describe them. **Make sure you provide the visualisation of results for the purpose of comparison.**

In [ ]:
# your code will go here

***Your analysis and dicussion will go here*** 

## Task 5: Social Implications - (20 Marks)

Finally, you will research and discuss two specific social implications of facial recognition technology:

1. **Bias and discrimination:** How can facial recognition technology lead to bias and discrimination? What steps can we take to address these issues?
2. **Law enforcement:** How is facial recognition technology being used by law enforcement agencies? What are the benefits and risks of using this technology in law enforcement?

***Discussion for Part 1*** 

***Discussion for Part 2***